## Multiple Chain 快速入门

Runnables 可以轻松地用来串联多个 Chains，使用 RunnablePassthrough 将输出同时传给多条后继链。

```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

本指南展示如何使用 Runnable 实现多个 AI 关于相同话题的辩论：

```
    输入话题
       |
       |
    原始观点
      / |\
     /  |  \
 正面论述| 反面论述
     \  | /
      \ |/
     最终总结
```

In [1]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 创建正面论证的处理链，列出关于基础回应的正面或有利的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的正面或有利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的反面或不利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出批评后生成最终回应"),
        ]
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [2]:
print(chain.invoke({"input": "房地产低迷"}))

关于房地产低迷现象的讨论，正反观点各有其合理性和深度。以下是对这两种观点的批评和最终回应：

### 批评

1. **市场调整与稳定**：
   - 虽然价格回归理性有利于降低泡沫风险，但在实际操作中，价格下跌可能导致购房者信心进一步下降，形成恶性循环。而且，清理不良资产的过程可能伴随大规模的失业和经济不稳定，造成更大的社会成本。

2. **购房机会**：
   - 买方市场虽然为购房者提供了更多选择，但在经济低迷背景下，购房者的购买力可能因收入下降而受到限制，未必能够真正享受到优惠政策的好处。

3. **投资多样化**：
   - 虽然资产多样化有助于降低风险，但这也可能使得房地产市场失去重要的投资资金，从而加剧市场的低迷局面。

4. **推动政策改革**：
   - 政策反思与调整的过程往往较为缓慢，短期内市场低迷可能导致政策滞后，进一步压制市场活力。

5. **社会关注与社区建设**：
   - 尽管社区建设可能得到强化，但在经济困难时期，资金和资源的短缺可能使得社区建设的质量和速度受到影响，无法真正改善居民的生活质量。

6. **环境与可持续发展**：
   - 尽管可持续发展受到关注，但在经济低迷时，开发商可能更倾向于削减成本，而非投资于环保和可持续项目。

### 最终回应

综合以上分析，房地产低迷现象确实是一个复杂的市场动态，既有正面机遇，也伴随着诸多挑战。虽然市场调整可能带来价格回归理性和购房机会，但短期内的经济不稳定和消费者信心不足却可能导致更深层次的问题。

在应对房地产低迷时，各方应当采取综合手段，既要关注市场的恢复与发展，也要重视社会问题的解决和政策的及时调整。政府应通过灵活的政策和激励措施来促进市场的健康发展，从而实现经济的长期稳定和可持续增长。同时，开发商和投资者也应关注市场的多样性和灵活性，以适应不断变化的需求和环境。

最终，房地产市场的复苏与发展需要多方共同努力，以实现经济、社会和环境的协调发展。


#### 流式输出

In [3]:
## chain 最终输出经过了 StrOutputParser 处理，所以可以直接输出流式输出 s
for s in chain.stream({"input": "全球经济"}):
    print(s, end="", flush=True)

在讨论全球经济时，综合正面和反面的观点可以帮助我们更全面地理解其复杂性和动态性。以下是对上述论点的批评与最终回应：

### 批评

1. **全球化与经济增长**：
   - 尽管全球化带来了经济增长，但其副作用也需要关注，尤其是对本地产业的冲击和失业问题。国家应加强对本地产业的保护与支持，避免过度依赖外部市场。

2. **不平等与经济分配**：
   - 财富不平等确实是一个严峻的问题，许多政策未能有效解决这一根本性挑战。政策制定者应更关注社会公平，通过税收和福利政策促进财富的合理分配。

3. **科技进步与经济转型**：
   - 虽然科技推动经济发展，但也加剧了就业市场的不平等。政府应积极推动再培训和技能提升项目，以帮助受影响的劳动者转型。

4. **气候变化与可持续发展**：
   - 短期内追求经济增长可能会对环境造成长期伤害，因此各国应在经济政策中更好地平衡发展与环境保护。

5. **地缘政治与经济安全**：
   - 地缘政治紧张局势对经济的影响不容忽视，国家应加强外交合作，避免经济脱钩导致的全球供应链脆弱。

6. **疫情后的经济复苏**：
   - 疫情带来的挑战需要长期策略来应对，政府应在刺激经济的同时，关注债务问题的可持续性，以防止未来的经济危机。

7. **数字经济的崛起**：
   - 数字鸿沟和隐私问题是数字经济面临的重要挑战，解决这些问题需要政策和技术的双重努力，确保技术的普惠性和安全性。

### 最终回应

全球经济是一个复杂的生态系统，既有机遇也有挑战。面对全球化带来的机遇，各国应采取积极的政策以促进经济增长，同时要谨慎管理其潜在的负面影响。只有通过国际合作、包容性政策、可持续发展和对科技转型的全面应对，才能创造一个更美好的全球经济环境。

我们应当承认，全球经济的未来取决于各国如何平衡经济增长、社会公平和环境保护的关系。政策制定者、企业和社会各界需要共同努力，积极应对挑战，抓住机遇，以实现更为繁荣和可持续的全球经济。

### Homework: 实现一个多链版本的代码生成，输入功能需求，输出2种（Python，Java）以上编程语言的代码实现。